># TensorFlow로 2층 CNN 구현하기

>## CNN의 전체적인 구성

- 합성곱과 풀링층을 각각 2층, 그리고 전결합층과 드롭아웃층을 포함한 CNN 구성
- 여기에서 사용한 층 수와 필터 수는 앞선 개발자들이 연구하여 얻은 값으로 참조
> a. 입력층 (28x28)  
> b. 합성곱층: 필터 수 32개 (28x28x32)  
> c. 풀링층: 2x2, 스트라이드 2 (14x14x32)  
> d. 합성곱층: 필터 수 64개 (14x14x64)  
> e. 풀링층: 2x2 스트라이드 2 (7x7x64)  
> f. 전결합층 (3136)  
> g. 드롭아웃: 50%  
> h. 출력층: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

___
>## 합성곱층 구현하기

- 앞서 합성곱층은 단순 예측을 위한 CNN으로 `tf.constant`를 사용하였지만, 이번에는 학습을 진행할 예정으로 `tf.Variable`로 구현
- `tf.random_normal`로 정규부포를 활용한 난수로 초기화
```python
# 첫번째 합성곱층: 5*5 필터, 1채널, 필터수 32
w1 = tf.Variable(tf.random_normal([5, 5, 1, 32]))
conv1 = tf.nn.conv2d(X_image,
                     w1,
                     strides = [1, 1, 1, 1],
                     padding = "SAME")
```
- 두번째 합성곱층은 입력값의 채널 수가 32로 변경
```python
# 두번째 합성곱층: 5*5 필터, 32채널, 필터수 64
w2 = tf.Variable(tf.random_normal([5, 5, 32, 64]))
conv2 = tf.nn.conv2d(conv1,
                     w2,
                     strides = [1, 1, 1, 1],
                     padding = "SAME")
```

- 위 과정은 `tf.layers.conv2d`를 사용하면 쉽게 구현 가능
- 가중치는 입력값의 shape에 따라 자동으로 생성
- 출력 shape도 `filters` 매개변수 값에 따라 자동 생성
- 또한 `tf.layers.conv2d`를 사용하면 합성곱 이후 활성화 함수를 적용하므로, 비선형성을 지닌 CNN에서 성능 향상을 기대할 수 있음 (페널티)
```python
# 첫번째 합성곱층: 5*5 필터, 1채널, 필터수 32
conv1 = tf.layers.conv2d(inputs = X_image,
                         filters = 32,
                         kernel_size = [5, 5],
                         padding = 'SAME',
                         activation = tf.nn.relu)
```

___
>## 풀링층 구현하기

- 풀링층도 `tf.layers`를 사용하면 간단하게 구현 가능
- 풀링층은 매개변수가 따로 없으므로, 편하게 구현
```python
pool1 = tf.layers.max_pooling2d(inputs = conv1,
                                pool_size = [2, 2],
                                strides = 2)
```

___
>## 전결합층과 드롭아웃층 구현하기

- 전결합층 부터는 '이미지 형태'가 아무 상관 없기 때문에 `tf.reshape`를 사용하여 일차원 벡터로 사용
- 이미지 크기는 두차례 2x2 풀링으로 28X28 > 14X14 > 7X7로 작아졌으나 필터 수가 64로 증가하여 3136 화소 값을 갖게 됨
```python
pool2_flat = tf.reshape(pool2,
                        [-1, 7*7*64])
dense = tf.layers.dense(inputs = pool2_flat,
                        units = 1024,
                        activation = tf.nn.relu)
dropout = tf.layers.dropout(inputs = dense,
                            rate = 0.5,
                            training = True)
```

___
>## 전체 코드

- 2층 CNN의 전체 코드는 아래와 같으며 정답률이 99.23%까지 개선 (전결합층 90~95% 수준)
- CNN은 학습에 시간이 매우 오래 걸리며(특히 GPU가 없는 경우), 에포크 수를 낮춰 사전에 예상 시간을 가늠해 보는 것이 효율적
- 또한, **Cloud ML Engine** 사용도 리소스 효율 차원에서 검토해 보는 것이 좋음

In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/")

batch_size = 50

with tf.Graph().as_default():
  X = tf.placeholder(tf.float32,
                    [None, 784],
                    name = 'X')
  y = tf.placeholder(tf.float32,
                    [None,],
                    name = 'y')
  
  X_image = tf.reshape(X, [-1, 28, 28, 1])
  
  # 첫번째 합성곱층
  conv1 = tf.layers.conv2d(inputs = X_image,
                       filters = 32,
                       kernel_size = [5, 5],
                       padding = 'SAME',
                       activation = tf.nn.relu)
  
  # 첫번째 풀링층
  pool1 = tf.layers.max_pooling2d(inputs = conv1,
                              pool_size = [2, 2],
                              strides = 2)
  
  # 두번째 합성곱층
  conv2 = tf.layers.conv2d(inputs = pool1,
                          filters = 64,
                          kernel_size = [5, 5],
                          padding = 'SAME',
                          activation = tf.nn.relu)
  
  # 두번째 풀링층
  pool2 = tf.layers.max_pooling2d(inputs = conv2,
                                 pool_size = [2, 2],
                                 strides = 2)
  
  # 전결합층
  pool2_flat = tf.reshape(pool2,
                         [-1, 7 * 7 * 64])
  dense = tf.layers.dense(inputs = pool2_flat,
                        units = 1024,
                        activation = tf.nn.relu)
  
  # 드롭아웃층
  dropout = tf.layers.dropout(inputs = dense,
                             rate = 0.5,
                             training = True)
  
  # 출력층
  logits = tf.layers.dense(inputs = dropout,
                          units = 10,
                          name = 'output')
  predict = tf.argmax(logits,
                      1)
  
  # 손실
  with tf.name_scope('calc_loss'):
    onehot_labels = tf.one_hot(indices = tf.cast(y,
                                                tf.int32),
                               depth = 10)
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels = onehot_labels,
                                                           logits = logits,
                                                           name = 'xentropy')
    loss = tf.reduce_mean(cross_entropy,
                         name = 'xentropy_mean')
    
  # 손실 최적화
  train_op = tf.train.AdamOptimizer(0.0001).minimize(loss)
  
  # 정답률 구하기
  with tf.name_scope('calc_accuracy'):
    correct_prediction = tf.equal(tf.argmax(logits,
                                           1),
                                 tf.argmax(onehot_labels,
                                          1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,
                                     tf.float32))
    
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    total_batch = int(mnist.train.num_examples // batch_size)
    
    for epoch in range(20):
      for step in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        _, loss_value = sess.run([train_op, loss],
                                feed_dict = {X: batch_xs, y: batch_ys})
        
      print('Step: %d, Loss %f' %(step, loss_value))
      
    # 테스트하기
    _a = sess.run(accuracy, feed_dict = {X: mnist.test.images, y: mnist.test.labels})
    
    print('Accuracy: %f' % _a)

Extracting ./mnist/train-images-idx3-ubyte.gz
Extracting ./mnist/train-labels-idx1-ubyte.gz
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Step: 1099, Loss 0.260841
Step: 1099, Loss 0.258715
Step: 1099, Loss 0.068271
Step: 1099, Loss 0.006807
Step: 1099, Loss 0.024814
Step: 1099, Loss 0.018018
Step: 1099, Loss 0.004972
Step: 1099, Loss 0.000446
Step: 1099, Loss 0.115407
Step: 1099, Loss 0.002886
Step: 1099, Loss 0.005884
Step: 1099, Loss 0.015561
Step: 1099, Loss 0.011737
Step: 1099, Loss 0.001439
Step: 1099, Loss 0.002101
Step: 1099, Loss 0.011948
Step: 1099, Loss 0.028400
Step: 1099, Loss 0.000694
Step: 1099, Loss 0.001600
Step: 1099, Loss 0.000020
Accuracy: 0.990300


- 위의 학습을 진행하는 데 약 한시간 가량의 시간 소요
- Cloud ML 엔진 사용을 검토 필요